<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/slideshow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }

!pip install moviepy ipywidgets
!apt-get install -y fonts-dejavu-core
!pip install tqdm
from tqdm import tqdm

In [ ]:
#@title ##**Upload images** { display-mode: "form" }
from google.colab import files, drive
import os
import shutil
import zipfile

upload_source = "PC" #@param ["PC", "Google Drive", "Google Drive archive"] {type:"string"}


input_folder = "/content/input/"
if os.path.exists(input_folder):
    shutil.rmtree(input_folder)
os.makedirs(input_folder)

if upload_source == "PC":
    print("Please select images to upload from your PC")
    uploaded_images = files.upload()
    image_filenames = []

    for filename, file_data in uploaded_images.items():
        file_path = os.path.join(input_folder, filename)
        with open(file_path, 'wb') as f:
            f.write(file_data)
        image_filenames.append(file_path)

elif upload_source == "Google Drive":
    drive.mount('/content/drive')
    folder_name = input("Please enter the name of the folder on Google Drive: ")

    folder_path = f'/content/drive/My Drive/{folder_name}'
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_name}' not found on Google Drive.")
    else:
        image_filenames = []
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                src_path = os.path.join(folder_path, filename)
                dest_path = os.path.join(input_folder, filename)
                shutil.copy(src_path, dest_path)
                image_filenames.append(dest_path)
        if image_filenames:
            print("Images copied to /content/input/:")
            for filename in image_filenames:
                print(filename)
        else:
            print("No images found in the specified folder.")

elif upload_source == "Google Drive archive":
    drive.mount('/content/drive')
    archive_name = input("Please enter the name of the archive on Google Drive (including extension, e.g. 'images.zip'): ")

    archive_path = f'/content/drive/My Drive/{archive_name}'
    if not os.path.exists(archive_path):
        print(f"Archive '{archive_name}' not found on Google Drive.")
    else:
        print(f"Found archive: {archive_name}")
        try:
            with zipfile.ZipFile(archive_path, 'r') as zip_ref:
                image_filenames = []
                for file_info in zip_ref.filelist:
                    if file_info.filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                        filename = os.path.basename(file_info.filename)
                        if filename:
                            dest_path = os.path.join(input_folder, filename)
                            with zip_ref.open(file_info) as source, open(dest_path, 'wb') as target:
                                shutil.copyfileobj(source, target)
                            image_filenames.append(dest_path)
                            print(f"Extracted: {filename}")
                if image_filenames:
                    print("\nSuccessfully extracted images to /content/input/")
                else:
                    print("No images found in the archive.")
        except Exception as e:
            print(f"Error processing archive: {e}")
            image_filenames = []


In [ ]:
#@title ##**Upload audio** { display-mode: "form" }
upload_source = "PC"  #@param ["PC", "Google Drive"] {type:"string"}

from google.colab import files, drive
import os

if upload_source == "PC":
    print("Please select an audio file to upload")
    uploaded_audio = files.upload()
    audio_filename = list(uploaded_audio.keys())[0]
    print(f"Selected audio file: {audio_filename}")

elif upload_source == "Google Drive":
    def upload_from_drive():
        drive.mount('/content/drive')

        # Show contents of MyDrive folder with numbering
        print("\nAvailable audio files in MyDrive:")
        base_path = '/content/drive/MyDrive'
        files_in_drive = [f for f in os.listdir(base_path) if f.endswith(('.mp3', '.wav', '.m4a'))]

        if not files_in_drive:
            print("No audio files found in MyDrive")
            return None

        # Show files with numbers
        for i, file in enumerate(files_in_drive, 1):
            print(f"{i}. {file}")

        # Get user selection
        while True:
            try:
                choice = input("\nEnter the number of the file you want to use (or just press Enter to exit): ")
                if not choice:
                    return None

                file_index = int(choice) - 1
                if 0 <= file_index < len(files_in_drive):
                    selected_file = files_in_drive[file_index]
                    full_path = os.path.join(base_path, selected_file)
                    if os.path.exists(full_path):
                        print(f"Selected file: {selected_file}")
                        return full_path
                    else:
                        print("File not found. Please try again.")
                else:
                    print("Invalid number. Please try again.")
            except ValueError:
                print("Please enter a valid number.")

    # Main logic for Google Drive
    audio_filename = upload_from_drive()
    if audio_filename:
        print(f"Full path: {audio_filename}")
    else:
        print("No audio file selected. Exiting.")
        # Handle the case where no audio file is selected
else:
    print("Invalid upload source selected.")

In [ ]:
#@title ##**Configuration** { display-mode: "form" }
#selected_effect = "crossfade"  #@param ["crossfade", "fade", "slide_left", "slide_right", "none"] {type:"string"}
#transition_duration = 1.0  #@param {type:"number"}
with_text_option = "Without text"  #@param ["With text", "Without text"] {type:"string"}
font_size = 38  #@param {type:"slider", min:10, max:100, step:1}
text_color = "#FFFFFF"  #@param {type:"string"}
import ipywidgets as widgets
from IPython.display import display

transition_effect = widgets.Dropdown(
    options=[
        ('No effect', 'none'),
        ('Crossfade', 'crossfade'),
        ('Fade in/out', 'fade'),
        ('Slide left', 'slide_left'),
        ('Slide right', 'slide_right'),
        ('Slide up', 'slide_up'),
        ('Slide down', 'slide_down')
    ],
    value='none',
    description='Transition effect:',
)

transition_duration = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Transition duration (sec):',
    continuous_update=False
)

display(transition_effect, transition_duration)

In [ ]:
#@title ##**Run** { display-mode: "form" }
from moviepy.editor import *
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from PIL import Image, ImageDraw, ImageFont
import os
import numpy as np
from tqdm import tqdm
import gc

def create_image_with_text(image_path, text, font_size, text_color):
    with Image.open(image_path).convert("RGB") as image:
        draw = ImageDraw.Draw(image)

        font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
        font = ImageFont.truetype(font_path, font_size)

        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        position = ((image.width - text_width) // 2, image.height - text_height - 10)
        draw.text(position, text, fill=text_color, font=font)

        temp_image_name = os.path.splitext(os.path.basename(image_path))[0] + '.jpg'
        temp_image_path = os.path.join("/tmp", f"temp_with_text_{temp_image_name}")
        image.save(temp_image_path, format='JPEG', quality=100)

    return temp_image_path


first_image = Image.open(image_filenames[0])
video_width, video_height = first_image.size
first_image.close()

video_size = (video_width, video_height)

selected_effect = transition_effect.value
transition_dur = transition_duration.value
add_text = with_text_option

audio_clip = AudioFileClip(audio_filename)
audio_duration = audio_clip.duration

num_images = len(image_filenames)
total_transition_time = (num_images - 1) * transition_dur
image_duration = (audio_duration - total_transition_time) / num_images

if image_duration <= 0:
    print("The audio duration is too short for the given number of images and transition durations")
else:
    output_filename = "slideshow_video.mp4"
    fps = 24

    writer = FFMPEG_VideoWriter(output_filename, video_size, fps=fps)

    try:
        prev_img_clip = None

        for i, filename in enumerate(tqdm(image_filenames, desc="Creating video", dynamic_ncols=True)):
            if add_text == 'With text':
                temp_image_path = create_image_with_text(filename, os.path.basename(filename), font_size, text_color)
                img_path = temp_image_path
            else:
                img_path = filename

            img_clip = ImageClip(img_path).set_duration(image_duration)

            img_clip = img_clip.resize(height=video_height)  # Изменяем высоту, ширина подстраивается
            img_clip = img_clip.on_color(size=video_size, color=(0, 0, 0), pos=('center', 'center'))

            for t in np.arange(0, image_duration, 1.0 / fps):
                frame = img_clip.get_frame(t)
                writer.write_frame(frame)

            img_clip.close()
            del img_clip
            gc.collect()

            if add_text == 'With text':
                os.remove(temp_image_path)

        writer.close()
        audio_clip.close()

        final_output = "final_" + output_filename
        os.system(f'ffmpeg -y -i {output_filename} -i {audio_filename} -c:v copy -c:a aac -strict experimental {final_output}')

    finally:
        if writer is not None:
            writer.close()
        audio_clip.close()
        gc.collect()

In [ ]:
#@title ##**Download video** { display-mode: "form" }
from google.colab import files

files.download(output_filename)

In [ ]:
#@title ##**Download video to google drive (optional)** { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/final_slideshow_video.mp4" "/content/drive/MyDrive/"